# Exploración de indicadores que reflejen la incidencia de Covid-19 en Venezuela usando fuentes de datos no oficiales

# GoFundMe - Limpieza y tratamiento ético de los datos
___

GoFundMe es una campaña de crowdfunding. El crowdfunding aprovecha el poder de las redes sociales e Internet para proporcionar a la gente medios para recaudar fondos, ayudar a otros a superar dificultades y lograr los objetivos a los que aspiran.

Este notebook de Jupyter contiene todo el código utilizado para el tratamiento de los datos extraídos de la campañas de GoFundMe. 

___


## Configuración

Configurar el entorno importando las bibliotecas con las que se van trabajar. 

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
from currency_converter import CurrencyConverter
c = CurrencyConverter()

Se importan los datos:

In [3]:
df = pd.read_pickle('../datos/GoFundMeData_Venezuela_Covid_covid19_covid-19_20210513.pkl') #versión final

# convertimos la columna tags: lista -> string
df = df.explode('tags')

# nos asegutamos que la columna_count tenga formato numerico
df['donation_count'] = pd.to_numeric(df['donation_count'])

# Todos los tags: 'salud' los incluimos en 'Medical, Illness & Healing'
df.loc[df['tags'] == 'salud', 'tags'] = 'Medical, Illness & Healing'

# filtramos fechas que no corresponden a nuestro rango de interés
filtro = df['launch_date'] > '2020-01-01'
df = df[filtro]

In [4]:
# leemos las columnas
df.columns

Index(['url', 'title', 'tags', 'current amount', 'total_amount', 'description',
       'created', 'launch_date', 'country', 'donation_count', 'is_charity'],
      dtype='object')

e observaS que las columnas que podrían contener información que podría comprometer la privacidad de las personas involucradas en las campañas son: *url, title, description*. En ese orden de ideas, eliminamos estas columnas de los datos.

In [5]:
df = df.drop(['url', 'title', 'description'], axis = 1)
df.head() #verificamos que no se muestra información comprometedora

,tags,current amount,total_amount,created,launch_date,country,donation_count,is_charity
0,"Medical, Illness & Healing","$17,550","$50,000","Created April 2, 2021",2021-04-02,US,174,false
1,Volunteer & Service,"$5,730","$6,000","Created April 13, 2020",2020-04-13,US,148,false
2,"Medical, Illness & Healing","$5,350","$5,000","Created April 25, 2020",2020-04-25,US,142,false
3,"Medical, Illness & Healing","$3,239","$8,000","Created April 6, 2021",2021-04-06,US,37,false
4,Animals & Pets,"€1,050","€1,800","Created July 5, 2020",2020-07-05,US,32,false


Ahora, hay que asegurarse que el índice que se usa en cada fila es único.

In [6]:
n = len(df.index)
indices = np.arange(n)
df = df.set_index(indices)

### Limpieza de divisas

Antes que todo, verifiquemos que los datos estén completos:

In [7]:
# sub df de datos con valores nulos para la variable 'total_amount'
df[pd.isnull(df['total_amount'])]

,tags,current amount,total_amount,created,launch_date,country,donation_count,is_charity


Vemos que no hace falta ningún valor. 

Algo de lo que nos percatamos es que el dinero no se solicita en la misma divisa. Para solucionar esto, se identifican los signos de todas las divisas presentes en nuestros datos. En este orden de ideas, se crea una función que recibe una lista/arreglo/serie de cantidades de dinero expresadas en diferentes divisas y devuelve la lista de símbolos de cada una de estas divisas:

In [8]:
def lista_signos(dinero):
    signos = []
    for string in dinero:
        signo = ''
        string = str(string)
        for char in string:
            if char.isnumeric():
                break
            else:
                signo += char
        if signo == '':
            signo = 'no registra'
        signos.append(signo) 
    return signos

Identificamos los símbolos para las cantidades de dinero en las columnas: *current amount*, *total_amount*

In [9]:
#current amount
signos_current = np.array(lista_signos(df['current amount']))

#total_amount
signos_total = np.array(lista_signos(df['total_amount']))

#veamos cuáles signos se encuentra en cada caso
print(set(signos_current), set(signos_total))

{'€', '$', 'kr', 'no registra', 'CHF', '£'} {'€', '$', 'kr', 'CHF', '£'}


Exploremos las filas que no registran valor en la columna *current amount*:

In [10]:
df[signos_current == 'no registra'].head(3)

,tags,current amount,total_amount,created,launch_date,country,donation_count,is_charity
59,"Medical, Illness & Healing","1,796",$,"Created July 29, 2020",2020-07-29,US,51,false
70,"Medical, Illness & Healing",140,$,"Created April 11, 2021",2021-04-11,US,10,false
74,"Medical, Illness & Healing",142,€,"Created February 18, 2021",2021-02-18,US,7,false


Para los demás filas, verifiquemos que los signos de ambas columnas concuerdan:

In [11]:
# filtro para las columnas en las que sí se registra el signo en la columna: current amount
filtro = np.array(signos_current) != 'no registra'

sum(signos_current[filtro] != signos_total[filtro]) # el resultado de esta suma es el número de filas con incongruencias

0

Vemos que en todas las demás filas concuerdan, por lo tanto, se asumirá que la divisa que se utiliza en la columna *current amount* es la misma que se usa en la columna *total_amount*. Con esta información, se modificará el DataFrame para que sea más fácil generar resultados. Los cambios que se planean hacer son los siguientes:
1. Agregar una columna en donde se señale la divisa que se usa en la campaña.
2. Modificar las columnas *current amount* y *total_amount* para que cada entrada corresponda alvalor numérico de los montos.
3. Agregar dos columnas: *current amount in dollars*, *total amount in dollars* con los valores monetarios en dolares.
4. Agregar otra columna: *percentage of completion*, que cuantifica cuanto ha avanzado la campaña.

Cabe mencionar, que para cumplir estos objetivos se usará la biblioteca `currency_converter`.

Empezamos con el primer punto:

In [12]:
monedas = []
for signo in signos_total:
    if signo == 'CHF':
        monedas.append('CHF')
    elif signo == 'kr':
        monedas.append('SEK')
    elif signo == '$':
        monedas.append('USD')
    elif signo == '£':
        monedas.append('GBP')
    elif signo == '€':
        monedas.append('EUR')

#agregamos la columna de códigos
df.insert(3, 'currency used', monedas)

Para el segundo punto, creamos una función:

In [13]:
def modificar_columna_dinero(DataFrame, nombre_columna, signos):
    col = df.columns.get_loc(nombre_columna)
    n = len(DataFrame.index)
    for i in range(n):
        valor = DataFrame.iloc[i, col]
        valor = valor.replace(signos[i], '').replace(',', '') #quitamos el signo y la coma
        if valor.isnumeric():
            valor = int(valor)
        else:
            valor = np.nan # sabemos que los único nan que estén presentes en los datos
                           # serán los que se agreguen en este paso.
        DataFrame.iloc[i, col] = valor

In [14]:
# con la función que creamos, modificamos las columnas
modificar_columna_dinero(df, 'current amount', signos_current)
modificar_columna_dinero(df, 'total_amount', signos_total)

En el tercer paso se utilizará la biblioteca `currency converter`. Para evitar ambigüedades se tomará como referencia el valor del dólar del día 3 de marzo de 2021, que es el último día disponible en la biblioteca:

In [15]:
current_dolares = np.array([])
total_dolares = np.array([])
n = len(df.index)
for i in range(n):
    # usamos la última fecha disponible en la biblioteca para hacer el cambio a dolares
    current_dolares = np.append(current_dolares, c.convert(df['current amount'].iloc[i], df['currency used'].iloc[i], 
                                     'USD', pd.to_datetime('2021-03-09'))) 
    total_dolares = np.append(total_dolares, c.convert(df['total_amount'].iloc[i], df['currency used'].iloc[i], 
                                   'USD', pd.to_datetime('2021-03-09')))

#agregamos las nuevas columnas:
df.insert(4, 'current amount in dollars', current_dolares)
df.insert(5, 'total amount in dollars', total_dolares)

Por por último, insertamos la columna: *percentage of completion*

In [16]:
df.insert(6, 'percentage of completion', (current_dolares/total_dolares)*100)

Visualicemos el resultado final:

In [17]:
df.head()

,tags,current amount,total_amount,currency used,current amount in dollars,total amount in dollars,percentage of completion,created,launch_date,country,donation_count,is_charity
0,"Medical, Illness & Healing",17550,50000,USD,17550.00,50000.00,35.100000,"Created April 2, 2021",2021-04-02,US,174,false
1,Volunteer & Service,5730,6000,USD,5730.00,6000.00,95.500000,"Created April 13, 2020",2020-04-13,US,148,false
2,"Medical, Illness & Healing",5350,5000,USD,5350.00,5000.00,107.000000,"Created April 25, 2020",2020-04-25,US,142,false
3,"Medical, Illness & Healing",3239,8000,USD,3239.00,8000.00,40.487500,"Created April 6, 2021",2021-04-06,US,37,false
4,Animals & Pets,1050,1800,EUR,1248.87,2140.92,58.333333,"Created July 5, 2020",2020-07-05,US,32,false


Visualicemos el resultado final:

In [18]:
df.head()

,tags,current amount,total_amount,currency used,current amount in dollars,total amount in dollars,percentage of completion,created,launch_date,country,donation_count,is_charity
0,"Medical, Illness & Healing",17550,50000,USD,17550.00,50000.00,35.100000,"Created April 2, 2021",2021-04-02,US,174,false
1,Volunteer & Service,5730,6000,USD,5730.00,6000.00,95.500000,"Created April 13, 2020",2020-04-13,US,148,false
2,"Medical, Illness & Healing",5350,5000,USD,5350.00,5000.00,107.000000,"Created April 25, 2020",2020-04-25,US,142,false
3,"Medical, Illness & Healing",3239,8000,USD,3239.00,8000.00,40.487500,"Created April 6, 2021",2021-04-06,US,37,false
4,Animals & Pets,1050,1800,EUR,1248.87,2140.92,58.333333,"Created July 5, 2020",2020-07-05,US,32,false


Finalmente, exportamos estos datos que son con los que se trabajará:

In [19]:
df.to_csv('../datos/datos_GoFundMe.csv') 

### Agradecimiento a colaboradores y proceso de webscrapping:

Los datos sobre las campañas de GoFundMe son producto de una colaboración con **@HarveyMaddocks** y pueden encontrarse en este [repositorio](https://github.com/HarveyMaddocks/gofundme_webscraper). Estos datos fueron colectados mediante técnicas de *webscrapping*. A continuación, se expondrán brevemente los pasos que se siguieron en este procedmiento:
1. Como es común en muchos sitios web, GoFundMe tiene un motor de búsqueda interno que permite al usuario consultar campañas de *crowfunding* a partir de palabras clave. Para usar esta herramienta de la mejor manera posible, se propusieron las palabras clave que nos pudieran conducir al mayor número de campañas de interés. En nuestro caso, las palabras usadas fueron: *Venezuela*, *Covid*.

2. Ahora, usando el formato de url del sitio web de GoFundMe, se crea la dirección correspondiente a la primera página de la búsqueda de las palabras clave escogidas en el punto anterior.

3. Posteriormente, usando el navegador virtual *Selenium*, compatible con el lenguaje de programación *python*, fue posible acceder a la página web de GoFundMe usando la url creada en el punto anterior. Esta herramienta permite además extraer el archivo HTML correspondiente a las páginas web.

4. Luego, se utilizó la biblioteca *Beautiful soup* para extraer los datos de los archivos HTML de las páginas web. Por ejemplo, al usar esta función sobre las páginas del resultado de búsqueda, se pudo crear una lista con todas las direcciones url de los sitios de las campañas que aparecían como resultado. 

5. Entonces, usando las bibliotecas *Selenium*, para acceder a las páginas web de las campañas, y *Beautiful soup*, para colectar datos; se recorrió la lista de direcciones url y se extrayeron los datos de interés para cada una de las campañas en esta página. En nuestro caso, la información que se colectó de cada campaña fue: 
    * La dirección *url* de la página donde se encuentra alojada la campaña.
    * El *título* de la campaña.
    * las *etiquetas (tags)*, o la categoria a la que pertence cada la campaña.
    * El *monto actual* recaudado por la campaña.
    * El *monto total* a recaudar o meta de recaudación.
    * La *descripción* de la campaña
    * La *fecha de lanzamiento*
    * El *país de origen* de la campaña.
    * El número de donaciones que ha recibido la campaña.
    * Y un columna indicando si la campaña se trata de caridad o no.

Estos datos se iban añadiendo, campaña a campaña     en una tabla de tal forma que cada fila era una       campaña diferente.

6. Por último, se itera este procedimiento sobre todas las páginas del resultado de búsqueda de las palabras clave selecionadas
